In [1]:
import quantdev.backtest as bts

#### Classic three factor model from 台股研究室

In [2]:
# three factor from 台股研究室

roe = bts.get_factor('roe')
mtm = bts.get_factor('mtm_3m')
pbr = bts.get_factor('股價淨值比', asc=False)
factor = bts.get_factor(roe+mtm+pbr)

In [37]:
tw3factor = bts.backtesting(factor>=0.99, rebalance='Q')
tw3factor.summary

,strategy,0050.TT
Annual return,24.07%,10.81%
Total return,7525.18%,727.84%
Max drawdown,-72.55%,-55.66%
Annual volatility,28.59%,19.21%
Sharpe ratio,0.84,0.56
Calmar ratio,0.33,0.19
beta,0.59,-


In [38]:
tw3factor._plot_equity_curve()

#### Adjustment on rebalance dates and stock pool

In [41]:
universe = (bts.get_data('稅後淨利')>=0) & (bts.get_data('收盤價')>=bts.get_data('收盤價').rolling(60).mean())

roe = bts.get_factor('roe', universe=universe)
mtm = bts.get_factor('mtm_3m', universe=universe)
pbr = bts.get_factor('股價淨值比', asc=False, universe=universe)
factor = bts.get_factor(roe+mtm+pbr)

In [42]:
adjusted_tw3factor = bts.backtesting(factor>=0.99, rebalance='QR')
adjusted_tw3factor.summary


# We made two key adjustments to improve the strategy:
#
# 1. Universe Selection:
#    - Added profitability filter (positive net income)
#    - Added technical filter (price above 60-day moving average)
#    These filters help focus on financially healthier companies with positive price momentum
#
# 2. Rebalance Timing:
#    - Changed from calendar quarters ('Q') to quarterly financial report release dates ('QR')
#    - This ensures we trade on actual financial data after it becomes publicly available
#    - Helps capture the market reaction to new fundamental information
#    
# The results show improved performance with higher returns (33.25% vs 22.79% annually)
# while maintaining similar risk metrics
# However, the large MDD is still a concern, we will try to improve it in the next version

,strategy,0050.TT
Annual return,33.25%,10.81%
Total return,33221.64%,727.84%
Max drawdown,-61.43%,-55.66%
Annual volatility,29.03%,19.21%
Sharpe ratio,1.1,0.56
Calmar ratio,0.54,0.19
beta,0.6,-


In [43]:
adjusted_tw3factor._plot_equity_curve()

#### Examining risk with MAE MFE analysis, try to improve it by adding stop loss

In [23]:
adjusted_tw3factor._plot_maemfe()

# as we can see, the maximum loss for loss trades at Q3 is 15.16%, which means that the loss exceed 13% is mostly cannot recover

In [34]:
# so we can add a stop loss at 15% to improve the strategy
stop_lossed_tw3factor = bts.backtesting(factor>=0.99, rebalance='QR', stop_loss=0.15, stop_at='intraday')
stop_lossed_tw3factor.summary

# the MDD is reduced to 47%, which is originally 61%, with the sacrifice of annual return from 33.25% to 27.24%.

,strategy,0050.TT
Annual return,27.24%,10.81%
Total return,12994.90%,727.84%
Max drawdown,-46.91%,-55.66%
Annual volatility,24.30%,19.21%
Sharpe ratio,1.1,0.56
Calmar ratio,0.58,0.19
beta,0.37,-


In [ ]:
stop_lossed_tw3factor._plot_maemfe()

# As can be seen from the GMFE/ MAE, the stop loss is effective, the maximum loss is reduced to ~20%, which is originally ~40%